# Data Aggregation and Group Operations

In [1]:
import pandas as pd 
import numpy as np

## 1) GroupBy mechanics

Creamos un dataframe de ejemplo, supongamos que son datos de diferentes cuentas de una empresa con dos etiquetas diferentes: *key1, key2*.

In [8]:
df = pd.DataFrame({'producto' : list('aabba'),
                   'vendedor' : ['Juan', 'Celia', 'Juan', 'Celia', 'Juan'],
                  'balance' : np.random.randn(5) * 10,
                   'income' : np.random.randn(5) + 2
                  })

df

,balance,income,producto,vendedor
0,-8.873985,1.641793,a,Juan
1,-17.206636,1.367402,a,Celia
2,1.810115,1.616797,b,Juan
3,8.150826,1.985183,b,Celia
4,21.473105,3.971585,a,Juan


Podemos comprobar cual es el balance medio y el income medio (las dos variables numéricas de nuestro dataframe)

In [7]:
df.mean()#

balance   -1.965770
income     1.723944
dtype: float64

Pero sólo esta información no nos da información sobre los diferentes productos. Podemos querer obtener el balance y los ingresos medios por producto (o por vendedeor!). Para ello podemos agrupar por variable: **groupby**.

In [11]:
means = df.groupby('producto').mean()
means

,balance,income
producto,,
a,-1.535839,2.326926
b,4.980471,1.800990


Podemos ver que hemos creado un nuevo dataframe con la media agrupada por tipo de producto.

In [18]:
type(means)

pandas.core.frame.DataFrame

También podemos acceder directamente a la variable que nos interese. Por ejemplo, si sólo queremos conocer el balance medio por producto:

In [19]:
mean_producto = df.groupby('producto')['balance'].mean()
mean_producto

producto
a   -1.535839
b    4.980471
Name: balance, dtype: float64

In [21]:
type(mean_producto)

pandas.core.series.Series

Hemos creado una serie y por tanto podemos acceder utilizando el label (index).

In [22]:
mean_producto['a']

-1.5358389260260712

In [23]:
means

,balance,income
producto,,
a,-1.535839,2.326926
b,4.980471,1.800990


Otra forma de acceder a la misma información desde el dataframe *means* (creado por **groupby**)

In [25]:
means['balance']['a']

-1.5358389260260712

Podemos agrupar utilizando más de una variable. Por ejemplo, agrupando primero por *producto * y luego por *vendedor*:

In [26]:
df.groupby(['producto', 'vendedor']).mean()

balance    income
producto vendedor                     
a        Celia    -17.206636  1.367402
         Juan       6.299560  2.806689
b        Celia      8.150826  1.985183
         Juan       1.810115  1.616797

Otra forma de acceder a algunas funciones reservadas como es **mean** (o **count**) es a través del método aggregate dentro de **groupby**:**agg**

In [30]:
df.groupby(['producto', 'vendedor']).agg(['mean', 'count'])

balance          income      
                        mean count      mean count
producto vendedor                                 
a        Celia    -17.206636     1  1.367402     1
         Juan       6.299560     2  2.806689     2
b        Celia      8.150826     1  1.985183     1
         Juan       1.810115     1  1.616797     1

In [31]:
df

,balance,income,producto,vendedor
0,-8.873985,1.641793,a,Juan
1,-17.206636,1.367402,a,Celia
2,1.810115,1.616797,b,Juan
3,8.150826,1.985183,b,Celia
4,21.473105,3.971585,a,Juan


Podemos añadir funciones arbitrarias al método agg() de objetos groupby:

In [36]:
#Functión para sumar el número de caracteres de una serie de strings.
def strseries(serie):
    return serie.str.len().sum()

In [43]:
df.groupby('producto')['vendedor'].agg(strseries)

producto
a    13
b     9
Name: vendedor, dtype: int64

Analicemos con atención el comando anterior:
1. **groupby** por variable *producto*.
2. selección de variable ['*vendedor*'] (que sabemos es de tipo string).
3. añandimos a nuestro **groupby** una funcion arbitraria sobre serie de strings.

Lo mismo de forma más compacta con lambda functions:

In [46]:
df.groupby('producto')['vendedor'].agg(lambda strseries: strseries.str.len().sum())

producto
a    13
b     9
Name: vendedor, dtype: int64

### 1.2) Iterating over groups

Podemos iterar sobre los objetos agrupados. Iterar sobre ellos produce tuples (key,group), por lo tanto podemos extraer nuestros datos por grupo.

Ejemplo:

In [65]:
for key, group in df.groupby('producto'):
    print("Tipo de producto: %s"% key)
    print("Datos de producto:\n %s"%group)

Tipo de producto: a
Datos de producto:
      balance    income producto vendedor
0  -8.873985  1.641793        a     Juan
1 -17.206636  1.367402        a    Celia
4  21.473105  3.971585        a     Juan
Tipo de producto: b
Datos de producto:
     balance    income producto vendedor
2  1.810115  1.616797        b     Juan
3  8.150826  1.985183        b    Celia


Estos datos los podemos transformar a listas de dataframes:

In [75]:
list(df.groupby('producto'))

[('a',      balance    income producto vendedor
  0  -8.873985  1.641793        a     Juan
  1 -17.206636  1.367402        a    Celia
  4  21.473105  3.971585        a     Juan),
 ('b',     balance    income producto vendedor
  2  1.810115  1.616797        b     Juan
  3  8.150826  1.985183        b    Celia)]

y producir diccionarios de dataframe:

In [76]:
cuentas=dict(list(df.groupby('producto')))

In [80]:
cuentas

{'a':      balance    income producto vendedor
 0  -8.873985  1.641793        a     Juan
 1 -17.206636  1.367402        a    Celia
 4  21.473105  3.971585        a     Juan,
 'b':     balance    income producto vendedor
 2  1.810115  1.616797        b     Juan
 3  8.150826  1.985183        b    Celia}

In [81]:
type(cuentas['a'])

pandas.core.frame.DataFrame

In [82]:
cuentas['a']['balance']

0    -8.873985
1   -17.206636
4    21.473105
Name: balance, dtype: float64

## 2) Data aggregation

Para esta parte vamos a descargar un archivo online:

In [83]:
import requests

url = 'https://raw.githubusercontent.com/wesm/pydata-book/1st-edition/ch08/tips.csv'
response = requests.get(url)

out_file = open('tips.csv', 'wb')
out_file.write(response.content)
out_file.close()

In [85]:
tips = pd.read_csv('tips.csv')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


#### Ejercicio

Obtener el porcentaje de propina y analizar la dependencia con las variables sexo y fumadores (*sex* y *smoker*). ¿Se puede ver alguna diferencia de comporamiento entre hombres/mujeres, y si son o no fumadores?

#### Solución

1. Añadir la variable deseada *tips_pct* en el dataframe original *tips*.
2. Agrupar por sexo y por si son fumadores.
3. Agregar variable estadísticas para interpretar los resultados.

In [104]:
#1. Creando variable para el porcentaje de tip
tips['tip_pct'] = np.round(100 * tips['tip'] / tips['total_bill'],decimals=1)
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,5.9
1,10.34,1.66,Male,No,Sun,Dinner,3,16.1
2,21.01,3.50,Male,No,Sun,Dinner,3,16.7
3,23.68,3.31,Male,No,Sun,Dinner,2,14.0
4,24.59,3.61,Female,No,Sun,Dinner,4,14.7


In [134]:
#2. Agrupamos.
grouped = tips.groupby(['sex','smoker'])

Podemos usar decribe para darnos datos estadísticos de nuestro DataFrameGrouped.

In [136]:
grouped['tip_pct'].describe()

count       mean       std  min     25%    50%     75%   max
sex    smoker                                                              
Female No       54.0  15.692593  3.648512  5.7  14.000  14.95  18.175  25.3
       Yes      33.0  18.215152  7.164588  5.6  15.200  17.40  19.800  41.7
Male   No       97.0  16.067010  4.187624  7.2  13.200  15.80  18.600  29.2
       Yes      60.0  15.281667  9.053260  3.6  10.225  14.10  19.150  71.0

O podemos utilizar agregaciones. Si queremos añadir funciones adicionales.
Nota: en este caso vamos a renombrar las colunmnas de la agregación!

In [128]:
#Maximo menos el mínimo
def peak_to_peak(s):
    return s.max() - s.min()

def rango_normal(s):
    return 4*s.std()#2 std por cada lado

In [137]:
#3. Agregamos variables estadisticas para entender mejor los datos.
grouped['tip_pct'].agg([('media','mean'), ('std dev','std'), 'count',('rango', peak_to_peak),('rango (95%)',rango_normal)])

media   std dev  count  rango  rango (95%)
sex    smoker                                                
Female No      15.692593  3.648512     54   19.6    14.594049
       Yes     18.215152  7.164588     33   36.1    28.658353
Male   No      16.067010  4.187624     97   22.0    16.750495
       Yes     15.281667  9.053260     60   67.4    36.213041

Comparando rango con rango (95%) parece claro que tenemos alguonos outlayers en el caso de fumadores masculinos... Miremos ahora los límites (min y max).

In [138]:
#3. Agregamos variables estadisticas para entender mejor los datos.
grouped['tip_pct'].agg([('media','mean'), ('std dev','std'),'min','max',('rango (95%)',rango_normal)])

media   std dev  min   max  rango (95%)
sex    smoker                                             
Female No      15.692593  3.648512  5.7  25.3    14.594049
       Yes     18.215152  7.164588  5.6  41.7    28.658353
Male   No      16.067010  4.187624  7.2  29.2    16.750495
       Yes     15.281667  9.053260  3.6  71.0    36.213041

Parece más fácil encontrar personas extremádamente generosas en el caso de los fumadores?

**Ejercicio: extraer los casos de tip superior al 40%.**

In [144]:
tips[tips['tip_pct']>40.0]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
172,7.25,5.15,Male,Yes,Sun,Dinner,2,71.0
178,9.60,4.00,Female,Yes,Sun,Dinner,2,41.7


**Ejercicio: repetir el análisis con las magnitudes absolutas de tip**

In [145]:
#3. Agregamos variables estadisticas para entender mejor los datos.
grouped['tip'].agg([('media','mean'), ('std dev','std'),'min','max',('rango (95%)',rango_normal)])

media   std dev   min   max  rango (95%)
sex    smoker                                             
Female No      2.773519  1.128425  1.00   5.2     4.513702
       Yes     2.931515  1.219916  1.00   6.5     4.879663
Male   No      3.113402  1.489559  1.25   9.0     5.958235
       Yes     3.051167  1.500120  1.00  10.0     6.000479

### 2.1 Pivot & unstack

Podemos pivotar nuestro grouped dataframe alrededor de la variable deseada con el comando **unstack**

In [146]:
stacked = df.groupby(['producto', 'vendedor']).mean()
stacked

balance    income
producto vendedor                     
a        Celia    -17.206636  1.367402
         Juan       6.299560  2.806689
b        Celia      8.150826  1.985183
         Juan       1.810115  1.616797

In [151]:
stacked.unstack?

In [148]:
stacked.unstack('vendedor')

balance              income          
vendedor      Celia      Juan     Celia      Juan
producto                                         
a        -17.206636  6.299560  1.367402  2.806689
b          8.150826  1.810115  1.985183  1.616797

De este modo podemos elegir cómo presentar los resultados según la variable a la que queramos dar más importancia.

In [162]:
stacked = df.groupby(['producto', 'vendedor']).mean()
stacked

balance    income
producto vendedor                     
a        Celia    -17.206636  1.367402
         Juan       6.299560  2.806689
b        Celia      8.150826  1.985183
         Juan       1.810115  1.616797

In [168]:
stacked.unstack('vendedor').unstack('balance')

         vendedor  producto
balance  Celia     a          -17.206636
                   b            8.150826
         Juan      a            6.299560
                   b            1.810115
income   Celia     a            1.367402
                   b            1.985183
         Juan      a            2.806689
                   b            1.616797
dtype: float64

Continuando con el ejemplo, podemos centrarnos directamente en uno de los vendedores.

In [171]:
df[df['vendedor']=='Celia'].groupby('producto').mean()

,balance,income
producto,,
a,-17.206636,1.367402
b,8.150826,1.985183


In [174]:
df[df['vendedor']=='Celia'].groupby('producto').mean().unstack()

         producto
balance  a          -17.206636
         b            8.150826
income   a            1.367402
         b            1.985183
dtype: float64

**pivot** hace pivtar la tabla entorno a la variable que deseamos.

In [176]:
df.pivot(columns='producto')

balance              income           vendedor       
producto          a         b         a         b        a      b
0         -8.873985       NaN  1.641793       NaN     Juan   None
1        -17.206636       NaN  1.367402       NaN    Celia   None
2               NaN  1.810115       NaN  1.616797     None   Juan
3               NaN  8.150826       NaN  1.985183     None  Celia
4         21.473105       NaN  3.971585       NaN     Juan   None

Esto muestra cada una de las entradas de nuestra tabla en relación al tipo de producto para cada una de las diferentes columnas de la tabla.

In [177]:
df

,balance,income,producto,vendedor
0,-8.873985,1.641793,a,Juan
1,-17.206636,1.367402,a,Celia
2,1.810115,1.616797,b,Juan
3,8.150826,1.985183,b,Celia
4,21.473105,3.971585,a,Juan


## Example: Filling missing values with group-specific values

In [242]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [284]:
df_us = pd.DataFrame(
    {
        'states': states,
        'market': ['East'] * 4 + ['West'] * 4,
        'data': [100,82,83,np.nan,20,30,np.nan,np.nan],
    }
)

In [285]:
df_us

,data,market,states
0,100.0,East,Ohio
1,82.0,East,New York
2,83.0,East,Vermont
3,NaN,East,Florida
4,20.0,West,Oregon
5,30.0,West,Nevada
6,NaN,West,California
7,NaN,West,Idaho


Podemos utilizar apply para aplicar una función sobre la serie.

In [287]:
df_us.groupby('market')['data'].apply(lambda x: x.fillna(x.mean()))

0    100.000000
1     82.000000
2     83.000000
3     88.333333
4     20.000000
5     30.000000
6     25.000000
7     25.000000
Name: data, dtype: float64

In [288]:
df_us2=df_us.copy()

In [289]:
df_us2['data']=df_us.groupby(['market'])['data'].apply(lambda x: x.fillna(x.mean()))

In [291]:
df_us2

,data,market,states
0,100.000000,East,Ohio
1,82.000000,East,New York
2,83.000000,East,Vermont
3,88.333333,East,Florida
4,20.000000,West,Oregon
5,30.000000,West,Nevada
6,25.000000,West,California
7,25.000000,West,Idaho


También podemos asignar valores según el valor de la variable utilizando groupby y un diccionario para la transformación.

In [293]:
fill_values = {'East': 10, 'West': 200}
fill_func = lambda g: g.fillna(fill_values[g.name])

df_us.groupby('market').apply(fill_func)

,data,market,states
0,100.0,East,Ohio
1,82.0,East,New York
2,83.0,East,Vermont
3,10.0,East,Florida
4,20.0,West,Oregon
5,30.0,West,Nevada
6,200.0,West,California
7,200.0,West,Idaho


## Pivot tables and Cross-tabulation